In [2]:
import os
import pickle
import argparse

import gensim
from gensim.models import word2vec

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import nltk
from nltk.tokenize import word_tokenize


from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.utils import to_categorical
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
from keras import backend as K
from keras.callbacks import EarlyStopping

from sklearn import preprocessing


Using TensorFlow backend.


In [3]:
def sliding_window(df,window_size,window_stride,input_features=None, output_features=None):
    allData = df[input_features].values.tolist()
    if output_features:
        outputData = [i[0] for i in df[output_features].values.tolist()]
    myArray = [[]]
    outputDataArray = [] 
    
    start=0
    for i in range(0,len(allData),window_stride):
        if i == 0:
            myArray = [allData[i:window_size+i]]
            if output_features != None:
                #outputDataArray.append(outputData[window_size+i][0])
                outputDataArray=[outputData[i+window_size-1]]
                #print(1,type(outputDataArray))
        else:
            myArray.append(allData[i:window_size+i])
            
            if window_size+i >= len(allData):
                if output_features != None:
                    #print(2,type(outputDataArray))
                    outputDataArray.append(outputData[len(allData)-1])
                break
            if output_features != None:
                    #outputDataArray.append(outputData[window_size+i][0])
                    #print(3,type(outputDataArray))
                    outputDataArray.append(outputData[i+window_size-1])

    if output_features == None:
            return (np.array(myArray))
    print(np.shape(np.array(myArray)))
    #return (np.array(myArray), outputDataArray)
    return (myArray, outputDataArray)

def mystratifiedOutputSampler(df, target):
    #a = {i:[] for i in df[target].unique()}
    xTIndexes = []
    minLen = None
    #print(a)
    for i in df[target].unique():
        #a[i] = df.index[df[target] != i].tolist()
        a = df.index[df[target] != i].tolist()
        print(i,len(a))
        if minLen == None or minLen > len(a):
            minLen = len(a)
    #for i in a.keys():
    print(minLen)
    for i in df[target].unique():
        if not xTIndexes:
                print(2)
                xTIndexes = list(np.random.choice(df.index[df[target] != i].tolist(), size=minLen, replace=False))
        else:
            #print(OnlyNeutal)
            xTIndexes = xTIndexes + list(np.random.choice(df.index[df[target] != i].tolist(), size=minLen, replace=False))

    return (xTIndexes)

In [4]:

def fileProcessor(csvFile,stockName):
    dateparse = lambda x: pd.datetime.strptime(x, '%d-%m-%Y')
    df = pd.read_csv(csvFile, parse_dates=['date'],date_parser=dateparse)
    #df = pd.read_csv("/datadrive/Sahil/code/GL/fewTrails/Datasets/GE.csv", parse_dates=['date'],date_parser=dateparse)


    df["date"]  = pd.to_datetime(df.date)
    #type(df["date"].iloc[0])
    df = df.sort_values(by="date")

    df['close_delta'] = 0
    df['close_direction'] = 0
    df['Stock_name'] = stockName
    for index in range(0,df.shape[0]):
        #print(index,df.iloc[index]['close'])
        if index == 0:
            df['close_delta'].iloc[index] =  (float(0))
            df['close_direction'].iloc[index] =  1
        #elif index <= df.shape[0]-1:
        else:
            df['close_delta'].iloc[index] = ((df['close'].iloc[index] - df['close'].iloc[index-1])/df['close'].iloc[index-1])*100
            if df['close_delta'].iloc[index] >= 1:
                df['close_direction'].iloc[index] = 2
            elif df['close_delta'].iloc[index] <= -1:
                df['close_direction'].iloc[index] = 0
            else:
                df['close_direction'].iloc[index] = 1
    return (df)
'''
fileCount = 0
import os
for root, dirs, files in os.walk("/datadrive/Sahil/code/GL/fewTrails/Datasets"):
    for file in files:
        if file.endswith(".csv"):
            print(os.path.join(root, file))
            if fileCount == 0:
                df = fileProcessor(os.path.join(root, file),file.split('.')[0])
            else:
                df = df.append(fileProcessor(os.path.join(root, file),file.split('.')[0]))
                
'''    
gCount=0
#df.apply(lambda x: featureTransform(x,df.shape[0]),axis=1)
#df.shape[0]
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
csvFile = "/datadrive/Sahil/code/GL/fewTrails/Datasets/ALL_clean.csv"
df = pd.read_csv(csvFile, parse_dates=['date'],date_parser=dateparse)

In [5]:
window_size = 1
input_cols = ['curr_ratio','tot_debt_tot_equity', 'oper_profit_margin','asset_turn','ret_equity','sentiment']

input_cols_scale = ['curr_ratio','tot_debt_tot_equity', 'oper_profit_margin','asset_turn','ret_equity',]

df = df.dropna().reset_index(drop=True)
#df_all.reset_index(drop=True)
mydf =  pd.DataFrame(preprocessing.scale(df[input_cols_scale]),columns=input_cols_scale)
mydf['sentiment'] = df['sentiment']
mydf['close_direction'] = df['close_direction']
mydf['Stock_name'] = df['Stock_name']
train_indexes = mystratifiedOutputSampler(mydf,'close_direction')
mydf = mydf.iloc[train_indexes]

x_train,y_train=sliding_window(mydf,window_size,1,input_cols,['close_direction'])
x_train = np.array(x_train)
print(np.shape(x_train))
for index,stock in enumerate(mydf['Stock_name'].unique()):
    print(stock)
    if index == 0:
        x_train,y_train=sliding_window(mydf[mydf['Stock_name'] == stock],window_size,1,input_cols,['close_direction'])
    else:
        x_,y_=sliding_window(mydf[mydf['Stock_name'] == stock],window_size,1,input_cols,['close_direction'])
        print(type(x_train),type(x_),np.shape(x_train))
        x_train = x_train + x_
        y_train = y_train + y_
x_train = np.array(x_train)
#y_train = np.array(y_train)

1 2693
0 7446
2 7175
2693
2
(8079, 1, 6)
(8079, 1, 6)
GE
(1148, 1, 6)
APPLE
(1093, 1, 6)
<class 'list'> <class 'list'> (1148, 1, 6)
IBM
(1126, 1, 6)
<class 'list'> <class 'list'> (2241, 1, 6)
MCD
(1058, 1, 6)
<class 'list'> <class 'list'> (3367, 1, 6)
BOEING
(1266, 1, 6)
<class 'list'> <class 'list'> (4425, 1, 6)
MICROSOFT
(1200, 1, 6)
<class 'list'> <class 'list'> (5691, 1, 6)
DISNEY
(1188, 1, 6)
<class 'list'> <class 'list'> (6891, 1, 6)


In [6]:
y_train1 = np.array(y_train)
y_train1 = np.array(y_train1).reshape((-1, 1))
#y_train = to_categorical(y_train)

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
#integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(y_train1)

In [7]:
np.shape(y_train)[1]
#onehot_encoder.transform(y_train1[0]).tolist()[0]
#for i in onehot_encoder.transform(y_train1):
#    print(i)

3

In [8]:
from sklearn.model_selection import train_test_split

def getBalancedSamples(x_train,proportionBYNonNeutal=.7,neutalProportionToOther=.7):
    #proportionBYNonNeutal = .7
    #neutalProportionToOther = .7
    AllWithAnyOtherThanNeutal = []
    OnlyNeutal=[]
    #for index,i in enumerate(x_train[1:500,:,:]):
    for index,i in enumerate(x_train[:,:,:]):
        #print()
        #print(i)
        #print(i[:,5])
        if 1 in i[:,5] or 2 in i[:,5] or 4 in i[:,5] or 5 in i[:,5]  :
            #print("TRUE")
            if not AllWithAnyOtherThanNeutal:
                AllWithAnyOtherThanNeutal = [index]
            else:
                AllWithAnyOtherThanNeutal.append(index)
        else:
            if not OnlyNeutal:
                print(2)
                OnlyNeutal = [index]
            else:
                #print(OnlyNeutal)
                OnlyNeutal.append(index)
    print (len(AllWithAnyOtherThanNeutal))
    print (len(OnlyNeutal))
    #AllWithAnyOtherThanNeutal[np.random.randint(0,100,size=20)]
    proportion = .7
    size = int(round(proportion*len(AllWithAnyOtherThanNeutal),0))
    print(size)
    #xTIndexesPart1 = [AllWithAnyOtherThanNeutal[i] for i in list(np.random.randint(0,100,size=size))]
    #xTIndexesPart2 = [OnlyNeutal[i] for i in list(np.random.randint(0,100,size=int(round(size*.7)))) ]
    #print(len(xTIndexesPart1),len(xTIndexesPart2))
    #xTIndexes = AllWithAnyOtherThanNeutal + OnlyNeutal
    
    #X_train, X_test, y_train, y_test = train_test_split(x_train[xTIndexes,:,:], y[xTIndexes], test_size=0.2)
    xTIndexesPart1 = list(np.random.choice(AllWithAnyOtherThanNeutal, size=size, replace=False))
    xTIndexesPart2 = list(np.random.choice(OnlyNeutal, size=int(round(size*.7)), replace=False))
    xTIndexes = xTIndexesPart1 + xTIndexesPart2
    print(len(xTIndexes))
    xTIndexesPart1 = list(np.random.choice(xTIndexes, size=len(xTIndexes), replace=False))
    #xTIndexes = np.sort(xTIndexes)
    return(xTIndexes)


In [9]:
df['close_direction'].unique()

array([1, 0, 2])

In [10]:
#y_train
train_indexes = getBalancedSamples(x_train)
X_train = x_train[train_indexes]
Y_train = y_train[train_indexes]

#X_train = x_train
#Y_train = y_train
Y_train

2
475
7604
332
564


array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [58]:
from numpy.random import seed
seed(1)
model = Sequential()
# input_shape = number of time-steps, number-of-features
model.add(LSTM(32,input_shape=(window_size,len(input_cols)),
               activation='relu', 
               inner_activation='hard_sigmoid', 
               return_sequences=True))
model.add(LSTM(32, activation='relu', recurrent_activation='hard_sigmoid'))
#model.add(Activation('sigmoid'))
model.add(Dropout(0.1))
#model.add(TimeDistributedDense(11))
#model.add(Dense(128))
#model.add(Dense(128,activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(output_dim=np.shape(y_train)[1], activation='softmax'))
#model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(32, activation="relu", recurrent_activation="hard_sigmoid", return_sequences=True, input_shape=(1, 6))`
/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`


In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 1, 32)             4992      
_________________________________________________________________
lstm_14 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_10 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_11 (Dropout)         (None, 16)                0         
__________

In [60]:
print('Train...')
model.fit(X_train, Y_train,
          batch_size=1,
          epochs=30,
          validation_data=(X_train, Y_train))
#score, acc = 


Train...
Train on 564 samples, validate on 564 samples
Epoch 1/30
564/564 [==============================] - 7s - loss: 1.0787 - val_loss: 1.0700
Epoch 2/30
564/564 [==============================] - 6s - loss: 1.0776 - val_loss: 1.0674
Epoch 3/30
564/564 [==============================] - 6s - loss: 1.0709 - val_loss: 1.0695
Epoch 4/30
564/564 [==============================] - 6s - loss: 1.0769 - val_loss: 1.0602
Epoch 5/30
564/564 [==============================] - 6s - loss: 1.0678 - val_loss: 1.0583
Epoch 6/30
564/564 [==============================] - 6s - loss: 1.0716 - val_loss: 1.0438
Epoch 7/30
564/564 [==============================] - 6s - loss: 1.0576 - val_loss: 1.0304
Epoch 8/30
564/564 [==============================] - 6s - loss: 1.0514 - val_loss: 1.0222
Epoch 9/30
564/564 [==============================] - 6s - loss: 1.0365 - val_loss: 1.0046
Epoch 10/30
564/564 [==============================] - 6s - loss: 1.0208 - val_loss: 1.0280
Epoch 11/30
564/564 [=============

In [61]:
score = model.evaluate(X_train, Y_train,batch_size=1)
print()
print('Test score:', score)
print(model.predict(X_train))
#print('Test accuracy:', confusion_matrix(Y_train,model.predict_classes(X_train)))

552/564 [============================>.] - ETA: 0s
Test score: 0.941588092085
[[  7.00256348e-01   2.55791783e-01   4.39519472e-02]
 [  4.51191701e-03   4.61902693e-02   9.49297845e-01]
 [  9.94346797e-01   5.55016613e-03   1.03055667e-04]
 ..., 
 [  1.55191019e-01   5.79142272e-01   2.65666693e-01]
 [  1.82182699e-01   5.41796923e-01   2.76020348e-01]
 [  1.86212851e-10   3.61490515e-06   9.99996424e-01]]


In [62]:
#model.predict(x_train)
a = model.predict_classes(X_train)

 32/564 [>.............................] - ETA: 0s

In [63]:
b = set(a)
a

array([0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1,
       0, 2, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1,
       0, 0, 1, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 2, 1, 2, 1, 2, 2, 0, 1, 1, 1,
       1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 1,
       2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 2, 0, 2, 2, 1, 2, 0, 2, 1, 2, 2, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0,
       1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1,
       1, 1, 1, 0, 1, 1, 2, 2, 1, 0, 1, 1, 1, 2, 1, 0, 1, 2, 2, 1, 1, 1, 1,
       1, 1,

In [64]:
y_train
#list(y_train1)
#onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.active_features_
labels = Y_train.dot(onehot_encoder.active_features_).astype(int)

In [65]:
accuracy = len(np.where(a == labels)[0].tolist())/len(labels)
print (accuracy)

0.6046099290780141


In [66]:
confusion_matrix(a.tolist(),labels.tolist())

## Older model with tanh and 128,128 LSTM layers
# 52% accuracy
#array([[ 55,  13,   4],
#       [ 63, 134,  77],
#       [ 31,  81, 103]])

array([[ 43,   3,   1],
       [102, 237,  99],
       [  7,  11,  61]])

In [735]:
confusion_matrix(a.tolist(),labels.tolist())

array([[ 55,  13,   4],
       [ 63, 134,  77],
       [ 31,  81, 103]])

In [736]:
output = pd.DataFrame(columns=['predicted','actual'])
output['predicted'] = a.tolist()
output['actual'] = labels.tolist()
output['Stock'] = list(mydf['Stock_name'].iloc[train_indexes])
output['Sentiment'] = list(mydf['sentiment'].iloc[train_indexes])

In [746]:
output.groupby(by="Stock")['predicted'].mean()

Stock
APPLE        1.098592
BOEING       1.402439
DISNEY       1.294872
GE           1.283951
IBM          1.225000
MCD          1.241379
MICROSOFT    1.219512
Name: predicted, dtype: float64

In [664]:
#mydf.iloc[train_indexes]

In [740]:
d.aggregate('actual')

TypeError: 'SeriesGroupBy' object is not callable

In [ ]:
ALL

In [ ]:
a=[[[]]]

In [ ]:
a[0] = ALL[0:5]

In [ ]:
a.append(ALL[1:6])

In [ ]:
a

In [ ]:
np.array(a)

In [ ]:
len(ALL)